In [2]:
!pip install implicit joblib pyarrow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 27.1 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
import random
import scipy.sparse as sparse
from implicit.als import AlternatingLeastSquares
import joblib
import os
import pyarrow.parquet as pq
import pyarrow as pa

random.seed(228)

# Глобальные переменные для модели и индексов
model = None
user_id_to_index = None
video_id_to_index = None

In [5]:
def load_model_and_indices(model_filepath):
    global model, user_id_to_index, video_id_to_index
    if os.path.exists(model_filepath):
        # Загрузка модели
        model = load_model(model_filepath)
        # Загрузка словарей индексов
        user_id_to_index = joblib.load('user_id_to_index.pkl')
        video_id_to_index = joblib.load('video_id_to_index.pkl')
        print(f"Модель и индексы загружены.")
    else:
        # Если модели нет, обучаем новую
        video_data = pd.read_parquet(video_data_path)
        interaction_matrix, user_id_to_index, video_id_to_index = create_interaction_matrix_in_batches(
            logs_path, video_data, batch_size
        )
        model = train_als_model(interaction_matrix)
        save_model(model, model_filepath)
        # Сохраняем словари индексов
        joblib.dump(user_id_to_index, 'user_id_to_index.pkl')
        joblib.dump(video_id_to_index, 'video_id_to_index.pkl')
        print(f"Модель обучена и сохранена в: {model_filepath}")

# Функция для загрузки данных пакетами из .parquet файла
def load_parquet_in_batches(parquet_path, batch_size):
    # Открываем Parquet файл
    parquet_file = pq.ParquetFile(parquet_path)
    num_row_groups = parquet_file.num_row_groups

    for rg in range(num_row_groups):
        # Читаем каждую группу строк (row group)
        table = parquet_file.read_row_group(rg)
        df = table.to_pandas()
        # Разбиваем на батчи внутри группы строк
        num_batches = len(df) // batch_size + 1
        for i in range(num_batches):
            batch_df = df[i*batch_size : (i+1)*batch_size]
            yield batch_df

# Функция для предобработки батча логов
def preprocess_logs_batch(logs_batch, video_data, video_id_to_index):
    # Объединяем логи с данными о видео по video_id
    logs_batch = logs_batch.merge(video_data[['video_id', 'v_duration']], on='video_id', how='left')

    # Преобразование watchtime в оценку неявной обратной связи (Implicit Score)
    def calculate_implicit_score(row):
        if pd.isna(row['v_duration']):
            return 0  # Если нет данных о длительности видео, считаем взаимодействие незначимым
        if row['v_duration'] > 5 * 60:  # Длительное видео (больше 5 минут)
            return 1 if row['watchtime'] > 0.25 * row['v_duration'] else 0
        else:  # Короткое видео (меньше 5 минут)
            return 1 if row['watchtime'] > 30 else 0

    # Применение функции к каждому ряду в DataFrame для вычисления implicit_score
    logs_batch['implicit_score'] = logs_batch.apply(calculate_implicit_score, axis=1)

    # Фильтрация только тех логов, где есть положительное взаимодействие
    logs_batch = logs_batch[logs_batch['implicit_score'] == 1]

    # Отображение user_id и video_id в индексы
    # Обновление словаря video_id_to_index
    for video_id in logs_batch['video_id'].unique():
        if video_id not in video_id_to_index:
            video_id_to_index[video_id] = len(video_id_to_index)

    logs_batch['video_index'] = logs_batch['video_id'].map(video_id_to_index)

    return logs_batch, video_id_to_index

# Функция для создания матрицы взаимодействий по батчам
def create_interaction_matrix_in_batches(parquet_path, video_data, batch_size):
    user_id_to_index = {}
    video_id_to_index = {}
    rows = []
    cols = []
    data = []

    # Используем функцию для загрузки данных пакетами
    logs_iterator = load_parquet_in_batches(parquet_path, batch_size)

    for logs_batch in logs_iterator:
        # Предобработка батча логов
        logs_batch, video_id_to_index = preprocess_logs_batch(logs_batch, video_data, video_id_to_index)

        # Обновление словаря user_id_to_index
        for user_id in logs_batch['user_id'].unique():
            if user_id not in user_id_to_index:
                user_id_to_index[user_id] = len(user_id_to_index)

        logs_batch['user_index'] = logs_batch['user_id'].map(user_id_to_index)

        # Добавление данных из батча в списки для создания матрицы
        rows.extend(logs_batch['video_index'])
        cols.extend(logs_batch['user_index'])
        data.extend(logs_batch['implicit_score'])

    # Создание разреженной матрицы взаимодействий
    interaction_matrix = sparse.coo_matrix(
        (data, (rows, cols)),
        shape=(len(video_id_to_index), len(user_id_to_index))
    )

    return interaction_matrix, user_id_to_index, video_id_to_index

# Остальные функции остаются без изменений

# Функция для обучения модели ALS
def train_als_model(interaction_matrix):
    model = AlternatingLeastSquares(
        factors=50,
        regularization=0.01,
        iterations=15,
        use_gpu=False  # Установите True, если у вас есть совместимый GPU
    )

    # Обучение модели (библиотека implicit ожидает матрицу item-user)
    model.fit(interaction_matrix.T)

    return model

# Функция для сохранения обученной модели
def save_model(model, filepath):
    joblib.dump(model, filepath)
    print(f"Модель сохранена по пути: {filepath}")

# Функция для загрузки обученной модели
def load_model(filepath):
    model = joblib.load(filepath)
    print(f"Модель загружена из: {filepath}")
    return model

# Функция для генерации рекомендаций
def generate_recommendations(model, video_id_to_index, current_user_index, watched_videos):
    # Получение факторов пользователя для текущего пользователя
    user_factors = model.user_factors[current_user_index]

    # Получение индексов просмотренных видео
    watched_videos_indices = [video_id_to_index[video] for video in watched_videos if video in video_id_to_index]

    # Исключение элементов, с которыми пользователь уже взаимодействовал
    known_items = set(watched_videos_indices)

    # Вычисление оценок для всех элементов (видео)
    scores = model.item_factors.dot(user_factors)

    # Исключение элементов, с которыми пользователь уже взаимодействовал
    video_scores = [(score, idx) for idx, score in enumerate(scores) if idx not in known_items]

    # Сортировка оценок видео по убыванию
    video_scores.sort(reverse=True)

    # Получение топ-10 индексов видео
    top_10_videos = [idx for score, idx in video_scores[:10]]

    # Отображение индексов обратно на video_id
    index_to_video_id = {idx: video_id for video_id, idx in video_id_to_index.items()}
    recommended_video_ids = [index_to_video_id[idx] for idx in top_10_videos]

    return recommended_video_ids

def run_recommendation_system(initial_ratings, watched_videos, current_user_id):
    global model, user_id_to_index, video_id_to_index

    # Проверяем, есть ли текущий пользователь в индексе
    if current_user_id in user_id_to_index:
        current_user_index = user_id_to_index[current_user_id]
    else:
        # Добавляем текущего пользователя в индекс
        current_user_index = len(user_id_to_index)
        user_id_to_index[current_user_id] = current_user_index
        # Обновление модели с новым пользователем
        # Добавляем нового пользователя в матрицу факторов пользователей
        model.user_factors = np.vstack([model.user_factors, np.random.normal(size=(model.user_factors.shape[1],))])
        # Не забываем сохранить обновленный индекс пользователей
        joblib.dump(user_id_to_index, 'user_id_to_index.pkl')

    # Обновление video_id_to_index для новых видео
    for video_id in (list(initial_ratings.keys()) + list(watched_videos)):
        if video_id not in video_id_to_index:
            video_id_to_index[video_id] = len(video_id_to_index)
            # Необходимо расширить матрицу факторов элементов
            model.item_factors = np.vstack([model.item_factors, np.random.normal(size=(model.item_factors.shape[1],))])
            # Сохраняем обновленный индекс видео
            joblib.dump(video_id_to_index, 'video_id_to_index.pkl')

    # Создаем списки для взаимодействий текущего пользователя
    current_user_video_indices = []
    current_user_scores = []

    def map_initial_rating(rating):
        if rating == 'like':
            return 2  # Более высокая оценка для лайков
        elif rating == 'dislike':
            return -1  # Отрицательная оценка для дизлайков
        else:
            return 0  # Нейтральное взаимодействие

    for video_id, rating in initial_ratings.items():
        video_index = video_id_to_index[video_id]
        score = map_initial_rating(rating)
        if score != 0:
            current_user_video_indices.append(video_index)
            current_user_scores.append(score)

    # Создание разреженного вектора для текущего пользователя
    current_user_interaction = sparse.coo_matrix(
        (current_user_scores, (current_user_video_indices, [0] * len(current_user_scores))),
        shape=(len(video_id_to_index), 1)
    )

    # Генерируем рекомендации
    recommendations = generate_recommendations(model, video_id_to_index, current_user_index, watched_videos)

    return recommendations

In [6]:
# Ввод данных
logs_path = '/content/drive/MyDrive/logs.parquet'
video_data_path = '/content/drive/MyDrive/video_stat.parquet'
model_filepath = 'als_model.pkl'
batch_size = 100000

# Загрузка модели и индексов при старте
load_model_and_indices(model_filepath)

/usr/local/lib/python3.10/dist-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 2 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()
/usr/local/lib/python3.10/dist-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 1.3625147342681885 seconds
  warnings.warn(


  0%|          | 0/15 [00:00<?, ?it/s]

Модель сохранена по пути: als_model.pkl
Модель обучена и сохранена в: als_model.pkl


In [13]:
# Вызов системы для получения рекомендаций FIRST TIME
initial_ratings = {
    "2f563d09-45ef-4e27-8da0-daa74235a2e7": "like",
    "1a3f527c-44b2-4f7e-9641-26f078edbf2e": "dislike",
    "973f24c0-15b2-434f-8740-4e2726f79c30": "neutral",
    "6cf5f321-77ca-4538-8b45-c2afc6c2f0d2": "like",
    "cf8913a2-d241-4204-91d5-f4fe02228c5b": "dislike",
    "3bf4bcb0-6372-4fd3-b9d9-0e7f79385259": "neutral",
    "0c9eb1fd-50f8-4a38-8030-7d9a939f2481": "like",
    "5cb3968d-f4ae-4c1a-a9eb-094a2e6fb3f2": "like",
    "cd0f11f2-0b8e-40c2-af27-373adc809a65": "neutral",
    "df123218-09fa-4ff4-9926-9167f8f61291": "like"
}
watched_videos = ["ed3f4c89-b1b0-46f5-bcbf-2d8e2832f663", "d818f9ef-1bdd-4e5f-8188-eb13dc5cbf6a"] + list(initial_ratings.keys())
current_user_id = "USER_HASH" # constant
model_filepath = "als_model.pkl" # learned model
recommendations = run_recommendation_system(
    initial_ratings, watched_videos, current_user_id
)
print("Рекомендованные видеo:", recommendations)

Рекомендованные видеo: ['568bf1a6-f1f1-4df2-8924-15dc4aba3af0', '56e0c337-a8b6-40c3-adc0-8e95b0b64e94', 'dad38bc3-de99-44f6-9675-2444e51493b2', '879cd972-e686-4894-8abd-de37a1aa33d2', 'ca775f67-64f5-409a-98f5-bd163f020997', 'd7fe6920-4ead-42ae-bbe3-e22deca936b6', '265cb8a0-6c8b-4256-a55d-e102c239886e', '9ed49d02-a300-4000-a086-352a170b0238', '47d47b93-1f8e-4de8-b441-1c7cc9e5c40d', '58083ffb-2ed8-47ef-a57d-86be82f4555b']


In [24]:
import random
ratings = {'like':0, 'dislike':0, 'neutral':0}

# циклы оценки подборки
for podborka in range(15):
    # сюда поступает оценка за новую подборку
    initial_ratings = dict(zip(recommendations, random.choices(list(ratings.keys()), weights=[4, 1, 2], k=10)))
    watched_videos = watched_videos + list(initial_ratings.keys())
    for key in list(ratings.keys()):
        ratings[key] += list(initial_ratings.values()).count(key)

    # выдача новой подборки
    recommendations = run_recommendation_system(initial_ratings, watched_videos, current_user_id)

In [32]:
print(ratings)

{'like': 110, 'dislike': 26, 'neutral': 64}

In [33]:
def presicion_colab(ratings):
    return ratings['like']/(ratings['like']+ratings['dislike']+(ratings['neutral']))

In [34]:
print(presicion_colab(ratings))

0.55